MAD Statistics

In [5]:
import os 
from scipy.io import loadmat
import numpy as np
from scipy import stats
from scipy.stats import mannwhitneyu, kruskal, wilcoxon, kstest, anderson
import math
import itertools 
import numpy as np
from scipy.stats import mannwhitneyu, kruskal, kstest, anderson
from fastdtw import fastdtw
import neurokit2 as nk
import matplotlib.pyplot as plt
import pandas as pd

#Place MAD data into SBS groups
data_dir = r"C:\Users\HP\Documents\JHU_Academics\Research\DT 6\NewPedAccel\VentilatedPatientData"
window_size = 100 # 100 is 1 second worth of time

lead_time = 10
slice_size_min = 15

Tag = "Nurse"
Tag = "Retro"

min = 5
min_stat = 10**-3
sr = 100 # Hz

In [6]:
# Function to calculate Cross-Correlation
def cross_correlation(signal1, signal2):
    return np.correlate(signal1, signal2, mode='full')

# Function to calculate Dynamic Time Warping (DTW)
def dtw(signal1, signal2):
    distance, path = fastdtw(signal1, signal2)
    return distance

# Function to calculate Cliff's Delta
def cliffs_delta(x, y):
    x, y = np.array(x), np.array(y)
    count = 0
    for i in x:
        for j in y:
            if i > j:
                count += 1
            elif i < j:
                count -= 1
    return count / (len(x) * len(y))

# Function to calculate Cohen's d
def cohens_d(x, y):
    diff = np.mean(x) - np.mean(y)
    pooled_std = np.sqrt((np.std(x) ** 2 + np.std(y) ** 2) / 2)
    return diff / pooled_std

# Function to calculate Hedges' g
def hedges_g(x, y):
    d = cohens_d(x, y)
    n1 = len(x)
    n2 = len(y)
    correction_factor = 1 - (3 / (4 * (n1 + n2 - 2) - 1))
    return d * correction_factor

# Function to compare signals
def compare_data(data_1, label_1, data_2, label_2, min_stat=0.05):
    significant_differences = []

    # Cross-Correlation
    cross_corr = cross_correlation(data_1, data_2)
    stat, p_value = mannwhitneyu(data_1, data_2)
    if p_value < 0.05 and p_value > min_stat:
        significant_differences.append(f"Cross-Correlation: {cross_corr}")
        print(f"Significant difference between {label_1} and {label_2} with Cross-Correlation: {cross_corr}")

    # Ensure input data is not empty
    if len(data_1) > 0 and len(data_2) > 0:
        try:
            # Convert inputs to 1D arrays (if needed)
            import numpy as np
            data_1 = np.array(data_1).flatten()
            data_2 = np.array(data_2).flatten()

            # Compute DTW distance
            dtw_distance = dtw(data_1, data_2)

            # Compute Mann-Whitney U test
            stat, p_value = mannwhitneyu(data_1, data_2)

            # Check for statistical significance
            if 0.05 > p_value > min_stat:
                significant_differences.append(f"DTW: {dtw_distance}")
                print(f"Significant difference between {label_1} and {label_2} with DTW: {dtw_distance}")
        
        except Exception as e:
            print(f"Error in DTW calculation: {e}")
    else:
        print("Error: One or both input data arrays are empty.")

    # Mann-Whitney U test
    stat, p_value = mannwhitneyu(data_1, data_2)
    if p_value < 0.05 and p_value > min_stat:
        significant_differences.append(f"Mann-Whitney U test: {p_value:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Mann-Whitney U test: {p_value:.4f}")

    # Kruskal-Wallis test
    stat, p_value = kruskal(data_1, data_2)
    if p_value < 0.05 and p_value > min_stat:
        significant_differences.append(f"Kruskal-Wallis test: {p_value:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Kruskal-Wallis test: {p_value:.4f}")

    # Weighted KS Test (requires scipy's ks_2samp)
    stat, p_value = kstest(data_1, data_2)
    if p_value < 0.05 and p_value > min_stat:
        significant_differences.append(f"Weighted KS test: {p_value:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Weighted KS test: {p_value:.4f}")

    # Kolmogorov-Smirnov test
    stat, p_value = kstest(data_1, data_2)
    if p_value < 0.05 and p_value > min_stat:
        significant_differences.append(f"Kolmogorov-Smirnov test: {p_value:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Kolmogorov-Smirnov test: {p_value:.4f}")

    # Anderson-Darling test
    result_1 = anderson(data_1)
    result_2 = anderson(data_2)
    p_value_1 = result_1.significance_level[0]
    p_value_2 = result_2.significance_level[0]
    if p_value_1 < 0.05 or p_value_2 < 0.05 and p_value_1 > min_stat and p_value_2 > min_stat:
        significant_differences.append(f"Anderson-Darling test: {p_value_1:.4f}, {p_value_2:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Anderson-Darling test: {p_value_1:.4f}, {p_value_2:.4f}")

    # Cliff's Delta
    delta = cliffs_delta(data_1, data_2)
    if abs(delta) > 0.474:  # Small, medium, large thresholds for Cliff's Delta
        significant_differences.append(f"Cliff's Delta: {delta:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Cliff's Delta: {delta:.4f}")

    # Cohen's d
    d = cohens_d(data_1, data_2)
    if abs(d) > 0.5:  # Cohen's d interpretation for large effect
        significant_differences.append(f"Cohen's d: {d:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Cohen's d: {d:.4f}")

    # Hedges' g
    g = hedges_g(data_1, data_2)
    if abs(g) > 0.5:  # Hedges' g interpretation for large effect
        significant_differences.append(f"Hedges' g: {g:.4f}")
        print(f"Significant difference between {label_1} and {label_2} with Hedges' g: {g:.4f}")

    return significant_differences


**tsfel features to calculate. Features can be found from MAD data or vector magnitude data**

In [7]:

#There is no error handling in place, the .mat file must exist
for patient in os.listdir(data_dir):
    # filter out non-directories
    patient_dir = os.path.join(data_dir, patient)
    if os.path.isdir(patient_dir):
        data_filepath_ecg = os.path.join(patient_dir, f'{patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN_ECG_SBSFinal.mat')
        
        if not os.path.isfile(data_filepath_ecg):
            print('skipping')
            continue
        

        ecg_data = loadmat(data_filepath_ecg)
        ecg_SBS = []
        ecg2 = ecg_data['ecg2']
        hrv_indices_final = pd.DataFrame()
        num_scores = np.shape(ecg2)[1]
        print(f'num scores: {num_scores}')

        #Collect all HRV features into a data frame
        for i in range((num_scores)):
            # Clean signal and Find peaks

            current_signal = ecg2[0][i].squeeze()
            if len(current_signal) == 0 or np.all(np.isnan(current_signal)):
                print('Skipping Empty Data')
                continue
                
            try:
                # Compute peaks and HRV indices
                peaks, info = nk.ecg_peaks(current_signal, sampling_rate=250, correct_artifacts=True)
                hrv_indices = nk.hrv(peaks, sampling_rate=250, show=False)
                hrv_indices.dropna(axis=1)
                hrv_indices_final = pd.concat([hrv_indices_final, hrv_indices], axis=0, ignore_index=True, sort=False)
                ecg_SBS.append(ecg_data['sbs_score'][0][i])

            except Exception as e:
                print(f"Error processing signal: {e}")
                continue

        # add SBS
        hrv_indices_final['SBS'] = ecg_SBS
        # Identify columns with no unique values
        cols_to_drop = [col for col in hrv_indices_final.columns if hrv_indices_final[col].nunique() == 1]
        # Drop the identified columns
        hrv_indices_final.drop(cols_to_drop, axis=1, inplace=True)

        for column in hrv_indices_final:
            SBS_neg3 = []
            SBS_neg2 = []
            SBS_neg1 = []
            SBS_zero = []            
            SBS_one = []
            SBS_two = []
            
            if column == 'SBS':
                continue

            feature_metrics = hrv_indices_final[column]
            print(f'Num rows: {len(feature_metrics)}')
            for i in range(len(feature_metrics)):
                ecg_SBS = hrv_indices_final['SBS'][i]
                feature_metric = feature_metrics[i]
                if ecg_SBS == -2:
                    SBS_neg2.append(feature_metric)
                elif ecg_SBS == -1:
                    SBS_neg1.append(feature_metric)
                elif ecg_SBS == 0:
                    SBS_zero.append(feature_metric)           
                elif ecg_SBS == 1:
                    SBS_one.append(feature_metric)
                elif ecg_SBS == 2:
                    SBS_two.append(feature_metric)
                else: 
                    continue
                
            # List of all SBS lists
            SBS_lists = [np.array(SBS_neg2).flatten(), np.array(SBS_neg1).flatten(), np.array(SBS_zero).flatten(), np.array(SBS_one).flatten(), np.array(SBS_two).flatten()]
            SBS_labels = ['SBS_neg2', 'SBS_neg1', 'SBS_zero', 'SBS_one', 'SBS_two']

            #Create 3 class version
            # SBS_lists = [np.concatenate([np.array(SBS_neg2).flatten(), np.array(SBS_neg1).flatten()]), np.concatenate([np.array(SBS_zero).flatten(),np.array(SBS_one).flatten()]), np.array(SBS_two).flatten()]
            # SBS_labels = ['group 1', 'group 2', 'group 3']

            # Compute all possible pairs of lists
            pairs = list(itertools.combinations(range(len(SBS_lists)), 2))
            assert(len(SBS_neg2) + len(SBS_neg1) + len(SBS_zero) + len(SBS_one) + len(SBS_two) > .5 * num_scores)

            # Display pairs and their corresponding labels
            for i, j in pairs:
                list_1 = SBS_lists[i]
                list_2 = SBS_lists[j]
                label_1 = SBS_labels[i]
                label_2 = SBS_labels[j]
                # if(len(list_1)/(slice_size_min * 60) > min and len(list_2)/(slice_size_min * 60) > min):
                if(len(list_1) > min and len(list_2) > min):
                    significant_differences = compare_data(list_1, label_1, list_2, label_2)


skipping
skipping
num scores: 11


c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-

Num rows: 11
list lengths: 0, 2
Not enough data
list lengths: 0, 2
Not enough data
list lengths: 0, 6
Not enough data
list lengths: 0, 1
Not enough data
list lengths: 2, 2
Not enough data
list lengths: 2, 6
Not enough data
list lengths: 2, 1
Not enough data
list lengths: 2, 6
Not enough data
list lengths: 2, 1
Not enough data
list lengths: 6, 1
Not enough data
Num rows: 11
list lengths: 0, 2
Not enough data
list lengths: 0, 2
Not enough data
list lengths: 0, 6
Not enough data
list lengths: 0, 1
Not enough data
list lengths: 2, 2
Not enough data
list lengths: 2, 6
Not enough data
list lengths: 2, 1
Not enough data
list lengths: 2, 6
Not enough data
list lengths: 2, 1
Not enough data
list lengths: 6, 1
Not enough data
Num rows: 11
list lengths: 0, 2
Not enough data
list lengths: 0, 2
Not enough data
list lengths: 0, 6
Not enough data
list lengths: 0, 1
Not enough data
list lengths: 2, 2
Not enough data
list lengths: 2, 6
Not enough data
list lengths: 2, 1
Not enough data
list lengths: 2,

c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2


Skipping Empty Data
Skipping Empty Data


...

Num rows: 111
list lengths: 0, 14
Not enough data
list lengths: 0, 33
Not enough data
list lengths: 0, 41
Not enough data
list lengths: 0, 23
Not enough data
list lengths: 14, 33
list lengths: 14, 41
Significant difference between SBS_neg1 and SBS_one with Cohen's d: 0.5814
Significant difference between SBS_neg1 and SBS_one with Hedges' g: 0.5732
list lengths: 14, 23
list lengths: 33, 41
list lengths: 33, 23
list lengths: 41, 23
Num rows: 111
list lengths: 0, 14
Not enough data
list lengths: 0, 33
Not enough data
list lengths: 0, 41
Not enough data
list lengths: 0, 23
Not enough data
list lengths: 14, 33
Significant difference between SBS_neg1 and SBS_zero with Cohen's d: -0.7428
Significant difference between SBS_neg1 and SBS_zero with Hedges' g: -0.7304
list lengths: 14, 41
Significant difference between SBS_neg1 and SBS_one with Cliff's Delta: -0.5784
Significant difference between SBS_neg1 and SBS_one with Cohen's d: -1.2378
Significant difference between SBS_neg1 and SBS_one with

c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


Skipping Empty Data


c:\Users\HP\anaconda3\envs\PHANGS\Lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2


KeyboardInterrupt: 